In [14]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.layers import *
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
BUFFER_SIZE = 60000
BATCH_SIZE = 256
(train_images, train_labels), (_, _) = tf.keras.datasets.mnist.load_data()
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1).astype('float32')
train_images = (train_images - 127.5) / 127.5  # Normalize the images to [-1, 1]
# Batch and shuffle the data
train_dataset = tf.data.Dataset.from_tensor_slices(train_images).shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [4]:
def make_discriminator_model():
    model = keras.models.Sequential()
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
    model.add(tf.keras.layers.Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(tf.keras.layers.Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
    model.add(tf.keras.layers.MaxPool2D(pool_size=(2,2), strides=(2,2)))
    model.add(tf.keras.layers.Dropout(0.25))
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(256, activation = "relu"))
    model.add(tf.keras.layers.Dropout(0.5))
    model.add(tf.keras.layers.Dense(10, activation = "softmax"))
    return model

In [5]:
discriminator_model = make_discriminator_model()

In [6]:
discriminator_model.compile(optimizer = "Adam" , loss = "categorical_crossentropy", metrics=["accuracy"])

In [9]:
discriminator_model.fit(train_images, tf.one_hot(train_labels, 10), epochs=2)

Epoch 1/2
1875/1875 [==============================] - 140s 75ms/step - loss: 0.3800 - accuracy: 0.8741
Epoch 2/2
1875/1875 [==============================] - 180s 96ms/step - loss: 0.0711 - accuracy: 0.9793


In [10]:
discriminator_model.save("CNN_V1_tfkerasmmodel")

INFO:tensorflow:Assets written to: CNN_V1_tfkerasmmodel/assets


# load model example

In [13]:
loaded_model = keras.models.load_model("CNN_V1_tfkerasmmodel")

import numpy as np
test_input = train_images[-1:]
np.testing.assert_allclose(
    loaded_model.predict(test_input), discriminator_model.predict(test_input)
)